In [1]:
!pip install transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.9 MB/s eta 0:00:00


In [2]:
import transformers
import torch
import torch.nn as nn

In [5]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier(
    [
    "What an awful movie.",
    "We had a joyius time and hope to have many more."
    ]
)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'NEGATIVE', 'score': 0.9997963309288025},
 {'label': 'POSITIVE', 'score': 0.9985551238059998}]

There are three major processes in a pipeline: <br/>

**Prepocessing (tokenizer), Model, Postprocessing**

*Preprocessing*

In [6]:
from transformers import AutoTokenizer

api = "distilbert-base-uncased-finetuned-sst-2-english"

tokenizer = AutoTokenizer.from_pretrained(api)

In [7]:
raw_input = [
    "What an awful movie.",
    "We had a joyius time and hope to have many more."
]

inputs = tokenizer(raw_input, padding=True, truncation=True, return_tensors="pt")

print(inputs)

{'input_ids': tensor([[ 101, 2054, 2019, 9643, 3185, 1012,  102,    0,    0,    0,    0,    0,
            0,    0,    0],
        [ 101, 2057, 2018, 1037, 6569, 4173, 2051, 1998, 3246, 2000, 2031, 2116,
         2062, 1012,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [10]:
from transformers import AutoModel

model = AutoModel.from_pretrained(api)

output = model(**inputs)
print(output.last_hidden_state.shape)

torch.Size([2, 15, 768])


In [11]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(api)

output = model(**inputs)

In [13]:
print(output.logits)

tensor([[ 4.7054, -3.7934],
        [-3.1836,  3.3547]], grad_fn=<AddmmBackward0>)


In [14]:
softmax = torch.nn.functional.softmax(output.logits, dim=-1)
print(softmax)

tensor([[9.9980e-01, 2.0366e-04],
        [1.4448e-03, 9.9856e-01]], grad_fn=<SoftmaxBackward0>)


In [16]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}